<a href="https://colab.research.google.com/github/weixincmu/community/blob/master/Google_ADK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-adk -q
!


In [ ]:
!pip install litellm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.8 MB/s eta 0:00:00


In [ ]:
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)


In [ ]:
from getpass import getpass

GEMINI=getpass("Enter your GEMINI API KEY: ")
os.environ["GOOGLE_API_KEY"]=GEMINI
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

Enter your GEMINI API KEY: ··········
Google API Key set: Yes


In [ ]:
model= "gemini-2.0-flash"

# Define Your Tool

In [ ]:
def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city (e.g., "Mumbai","Chennai","Delhi").

    Returns:
        dict: A dictionary containing the weather information.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'report' key with weather details.
              If 'error', includes an 'error_message' key.
    """
    # Best Practice: Log tool execution for easier debugging
    print(f"--- Tool: get_weather called for city: {city} ---")

    city_normalized = city.lower().replace(" ", "")  # Basic input normalization

    mock_weather_db = {
        "delhi": {"status": "success", "report": "The weather in Delhi is sunny with a temperature of 35°C."},
        "mumbai": {"status": "success", "report": "It's humid in Mumbai with a temperature of 30°C."},
        "bangalore": {"status": "success", "report": "Bangalore is experiencing light showers and a temperature of 22°C."},
        "kolkata": {"status": "success", "report": "Kolkata is partly cloudy with a temperature of 29°C."},
        "chennai": {"status": "success", "report": "It's hot and humid in Chennai with a temperature of 33°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}


# Example usage
print(get_weather("Mumbai"))


--- Tool: get_weather called for city: Mumbai ---
{'status': 'success', 'report': "It's humid in Mumbai with a temperature of 30°C."}


# Define the Agent

In [ ]:
AGENT_MODEL=model
weather_agent=Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL,
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. Your primary goal is to provide current weather reports. "
                "When the user asks for the weather in a specific city, "
                "you MUST use the 'get_weather' tool to find the information. "
                "Analyze the tool's response: if the status is 'error', inform the user politely about the error message. "
                "If the status is 'success', present the weather 'report' clearly and concisely to the user. "
                "Only use the tool when a city is mentioned for a weather request.",
    tools=[get_weather],

)

print(f"Agent '{weather_agent.name}' created using model '{AGENT_MODEL}'.")

Agent 'weather_agent_v1' created using model 'gemini-2.0-flash'.


# Setup Runner and Session Services

In [ ]:
# @title Setup Session Service and Runner
# ---Session Management ---
# Key Concept: SessionService stores conversation history & state.
# InMemorySessionService is  simple, non-persistent storage for this tutorial.

session_service=InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME="weathertutorial_app"
USER_ID="user_1"
SESSION_ID="session_001"

# Create the specific session where the conversation will happen
session=session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")


# ---Runner ---
# Key Concept: Runner orchestrates the agent execution loop.

runner=Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service
)

print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='weathertutorial_app', User='user_1', Session='session_001'
Runner created for agent 'weather_agent_v1'.


# Interact with the Agent

In [ ]:
# @title Define Agent Interaction Function
import asyncio
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

# Run the Conversation

In [ ]:
# @title Run the Initial Conversation

# # We need an async function to await our interaction helper
# async def run_conversation():
#     await call_agent_async("What is the weather like in Mumbai")
#     await call_agent_async("How about Delhi?") # Expecting the tool's error message
#     await call_agent_async("Tell me the weather in CHennai")

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: What is the weather like in Mumbai


--- Tool: get_weather called for city: Mumbai ---
<<< Agent Response: It's humid in Mumbai with a temperature of 30°C.


>>> User Query: How about Delhi?


--- Tool: get_weather called for city: Delhi ---
<<< Agent Response: The weather in Delhi is sunny with a temperature of 35°C.


>>> User Query: Tell me the weather in CHennai


--- Tool: get_weather called for city: Chennai ---
<<< Agent Response: It's hot and humid in Chennai with a temperature of 33°C.



In [ ]:
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash".
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"
print("\nEnvironment configured.")
